In [1]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import sys

sys.path.append("../..")

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gp
import seaborn as sns

import src.datasets.loading.ookla
import src.datasets.loading.statcan
import src.datasets.joins

from src.config import DATA_DIRECTORY
import warnings
warnings.filterwarnings('ignore')

In [342]:
das_info = gp.read_file(DATA_DIRECTORY / "hackathon" / "geometry.gpkg", driver="GPKG")
speed_data = pd.read_csv(DATA_DIRECTORY / "hackathon" / "speeds.csv")
speed_data.DAUID = speed_data.DAUID.astype(str)
full_data = pd.merge(das_info, speed_data)
full_data["date"] = pd.to_datetime(
    full_data["year"].astype(str)
    + "-"
    + ((full_data["quarter"] - 1) * 3 + 1).astype(str)
    + "-01"
)
# full_data['avg_d_kbps'] /= 1000000
# full_data['avg_u_kbps'] /= 1000000

In [3]:
full_data = pd.merge(das_info, speed_data)
full_data["date"] = pd.to_datetime(
    full_data["year"].astype(str)
    + "-"
    + ((full_data["quarter"] - 1) * 3 + 1).astype(str)
    + "-01"
)
# full_data['avg_d_kbps'] /= 1000
# full_data['avg_u_kbps'] /= 1000

In [281]:
unique_values = full_data['DAUID'].unique()
print(len(unique_values))
# print(unique_values)
df = pd.read_csv("./stat_result.csv")
print(len(df['Dissemination Area ID'].unique()))
# print(full_data['PCNAME'])
# datac = calgary_avg_d_kbps[:18].set_index('date', inplace=False)

56589
55608


In [343]:
full_data1 = [full_data[full_data['conn_type'] == 'fixed'],full_data[full_data['conn_type'] == 'mobile']]

In [9]:
if df[df['Dissemination Area ID']==48010135]['Meets 2022 Speed Goal'].any():
    print("yes!")
new_df = df[df['Province']=='Alberta']
new_df = new_df[new_df['Meets 2022 Speed Goal']==0]
print(len(new_df))

2243


In [10]:
selected_one = full_data[full_data['PCNAME'] == 'Athabasca']
# datac = selected_one['avg_d_kbps'].set_index('date', inplace=False).dropna(how='any')
datac = selected_one.set_index('date')['avg_d_kbps'].dropna(how='any')
print(datac.index[-1].date())
# .set_index('date', inplace=False)

2023-04-01


In [344]:
print(full_data1)

[            DAUID PRUID                                             PRNAME  \
0        10010244    10  Newfoundland and Labrador / Terre-Neuve-et-Lab...   
2        10010244    10  Newfoundland and Labrador / Terre-Neuve-et-Lab...   
4        10010244    10  Newfoundland and Labrador / Terre-Neuve-et-Lab...   
6        10010244    10  Newfoundland and Labrador / Terre-Neuve-et-Lab...   
8        10010244    10  Newfoundland and Labrador / Terre-Neuve-et-Lab...   
...           ...   ...                                                ...   
2263550  59530210    59            British Columbia / Colombie-Britannique   
2263552  59530210    59            British Columbia / Colombie-Britannique   
2263554  59530210    59            British Columbia / Colombie-Britannique   
2263556  59530210    59            British Columbia / Colombie-Britannique   
2263558  59530210    59            British Columbia / Colombie-Britannique   

        CDUID              CDNAME CDTYPE   CCSUID             

In [346]:
labels = df['Dissemination Area ID']
# print(labels)
res_down = [0]*len(labels)
res_up = [0]*len(labels)
#select order
def get_pq(data):
    #AIC
    AIC = sm.tsa.arma_order_select_ic(data, max_ar=7, max_ma=7, ic='aic')['aic_min_order']
    #BIC
    BIC = sm.tsa.arma_order_select_ic(data, max_ar=7, max_ma=7, ic='bic')['bic_min_order']
    # print('the AIC is{},\nthe BIC is{}\n'.format(AIC,BIC))
    return AIC
#predict
def predict(data, order, start, end, freq=None):
        model = ARIMA(data, order=order, freq=freq)
        result = model.fit()
        # pred = result.predict(start, end, dynamic=True, typ='levels')
        pred = result.forecast(10)
        return pred
for i,full_data in enumerate(full_data1):
    for index, dauid in enumerate(labels[5742:]):
        if df[df['Dissemination Area ID']==dauid]['Meets 2022 Speed Goal'].any():
            res_down[index] = 0
            res_up[index] = 0
            continue
        selected_one = full_data[full_data['DAUID'] == str(dauid)]
        # print(index)
        download = selected_one.set_index('date')['avg_d_kbps'].dropna(how='any')
        # print(download)
        # print("---------")
        upload = selected_one.set_index('date')['avg_u_kbps'].dropna(how='any')
        if download.shape[0] <4:
            res_down[index] = -1
        else:    
            try:
                pred_month = predict(download, (2, 1, 2), download.index[-1].date().strftime('%Y-%m-%d'), '2026-10-01')
            except:
                try:
                    pred_month = predict(download, (2, 1, 3), download.index[-1].date().strftime('%Y-%m-%d'), '2026-10-01')
                except:
                    pred_month = predict(download, (3, 1, 2), download.index[-1].date().strftime('%Y-%m-%d'), '2026-10-01')
            # print(name)
            res_down[index] = np.average(pred_month[pred_month.index[-4:]])
        # if pred_month[pred_month.index[-1]] < 50:
        #     unfullfilled.append(name)
        #     continue
        # [p,q] = get_pq(upload.diff(periods=1)[1:])
        if upload.shape[0] <4:
            res_up[index] = -1    
        else:
            try: 
                pred_month = predict(upload, (2, 1, 2), upload.index[-1].date().strftime('%Y-%m-%d'), '2026-10-01')
            except:
                try:
                    pred_month = predict(upload, (3, 1, 2), upload.index[-1].date().strftime('%Y-%m-%d'), '2026-10-01')
                except:
                    pred_month = predict(upload, (2, 1, 3), upload.index[-1].date().strftime('%Y-%m-%d'), '2026-10-01')
                
            res_up[index] = np.average(pred_month[pred_month.index[-4:]])
    csv_file = f"my_list_d_{i}.csv"
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        for item in res_down:
            writer.writerow([item])
    
    csv_file = f"my_list_u_{i}.csv"
    
    # 使用 csv 模块打开 CSV 文件进行写入
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        for item in res_up:
            writer.writerow([item])            
    # if pred_month[pred_month.index[-1]] < 10:
    #     unfullfilled.append(name)
    #     continue
    # fullfilled.append(name)
        
    

LinAlgError: LU decomposition error.

In [345]:
i = 3
ff = f"my_list_u_{i}.csv"
print(ff)

my_list_u_3.csv


In [337]:
# print(len(res_u))
print(np.flatnonzero(res_down)[-1])
print(np.flatnonzero(res_up)[-1])
length = min (np.flatnonzero(res_up)[-1],np.flatnonzero(res_down)[-1])
# length = len(res_down)
print(length)

5741
5741
5741


In [340]:
import csv

# print(new_u)
res_d = res_down[:length]
res_u = res_up[:length]
# new_d = res_d+res_down[:length]
# new_u = res_u+res_up[:length]
# res_d = new_d+res_down[:length]
# res_u = new_u+res_up[:length]
csv_file = 'my_list_d.csv'

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    for item in res_d:
        writer.writerow([item])

csv_file = 'my_list_u.csv'

# 使用 csv 模块打开 CSV 文件进行写入
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    for item in res_u:
        writer.writerow([item])
    
ll = len(res_d)
print(ll)
# print(np.flatnonzero(res_down)[-1])
# print(new_d)

5741


In [328]:
for i in range(len(new_d)):
    if (df['Meets 2022 Speed Goal'][i] == 1) and (new_d[i] !=0 or new_u[i] !=0):
        print("eee")
        break

In [271]:
rres_d = res_d
rres_u = res_u
real_res_d = rres_d[:593] + [0] + rres_d[593:801]+rres_d[802:]
real_res_u = rres_u[:593] + [0] + rres_u[593:801]+rres_u[802:]

In [329]:
# n=826
# print(rres_d[5820:5840])
# print(df['Meets 2022 Speed Goal'][5836])
# print(res_down)